In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

In [2]:
!ls ../

Makefile         crawl            docker           requirements.txt
README.md        data             model            scripts
config           deepseadream     notebook         tests


In [2]:
PATH = "../data/train_fish/"
ALB = "ALB"
BET = "BET"
DOL = "DOL"
LAG = "LAG"
NoF = "NoF"
OTHER = "OTHER"
SHARK = "SHARK"
YFT = "YFT"

In [3]:
import os
for path, di, file  in os.walk(PATH):
    CLASS = di
    break
NUM_CLASSES = len(CLASS)
IMAGE_SIZE = 256
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE*3
learning_rate = 1e-4
max_steps = 200
BATCH_SIZE = 100

In [ ]:
# def inference(inputs,reuse=False):
#     reshaped = tf.reshape(inputs,[inputs.get_shape()[0].value,-1])
    
#     with

In [4]:
# これらコードは、http://kivantium.hateblo.jp/entry/2015/11/18/233834から得られる
def inference(images_placeholder,keep_prob):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)
    def bias_variable(shape):
        initial = tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
    def conv2d(x,W):
        return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')
    x_image = tf.reshape(images_placeholder,[-1,28,28,3])
    with tf.name_scope('conv1') as scope:
        W_conv1 = weight_variable([5,5,3,32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)

    with tf.name_scope('conv2') as scope:
        W_conv2 = weight_variable([5,5,32,64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_conv1,W_conv2) + b_conv2)


    
    #全結合
    with tf.name_scope('zenketsu1') as scope:
        W_fc1 = weight_variable([7*7*64,1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_conv2,[-1,7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
        #dropout
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
        
    with tf.name_scope('zenketsu2') as scope:
        W_fc2 = weight_variable([1024,NUM_CLASSES])
        b_fc2 = bias_variable([NUM_CLASSES])
    
    with tf.name_scope('softmax') as scope:
        y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    
    return y_conv

In [5]:
def loss(logits,labels):
    cross_entropy = -tf.reduce_sum(labels*tf.log(logits))
#     tf.scalar_summary("cross_entropy",cross_entropy)
    return cross_entropy

In [6]:
def training(loss,learning_rate):
    train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)
    return train_step

In [7]:
# file load
def load_files(path):
    data = []
    for pa,cl,files in os.walk(path):
        for n_c, p in enumerate(pa):
            for file in files:
                filepath = os.path.join(pa,file)
                data.append([filepath,n_c])
    return data
DATA = load_files(PATH)
import csv
with open("../data/meta.csv",'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    writer.writerow(DATA)

In [16]:
# DATA = [(d[1],d[0]) for d in DATA]
import random
random.shuffle(DATA)
num_train = int(len(DATA) * 0.8)
train = DATA[:num_train]
test = DATA[num_train:]
# for d in DATA:
#     jpeg_r = tf.read_file(d[0])
#     image = tf.image.decode_jpeg(jpeg_r,channels=3)
#     image = tf.image.resize_images(image,[256,256])
#     image = tf.image.per_image_standardization(image)
    
#     label = tf.one_hot()

# def load_data(csvpath,batch_size,image_size,class_count,shuffle=False,min_after_dequeue=1000):
#     queue = tf.train.string_input_producer([csvpath],shuffle=shuffle)
#     reader = tf.TextLineReader()
#     key,value = reader.read(queue)
#     imagepath, label = tf.decode_csv(value[['imagepath'],[0]])

In [9]:
def train_batch(data_list):
    labels,paths = [],[]
    for data in data_list:
        labels.append(data[0])
        paths.append(data[1])
    queue = tf.train.slice_input_producer([labels,paths])
    label = queue[0]


    jpeg = tf.read_file(queue[1])
    image = tf.image.decode_jpeg(jpeg,channels=3)
    image = tf.image.resize_images(image,[256,256])

    image = tf.image.per_image_standardization(image)

    return tf.train.shuffle_batch([image,label],batch_size=32,
                                  capacity = len(data_list) * 2 + 3* 32,
                                 min_after_dequeue = len(data_list) *2)

In [10]:
def test_batch(data_list):
    labels,paths = [],[]
    for data in data_list:
        labels.append(data[0])
        paths.append(data[1])
    queue = tf.train.slice_input_producer([labels,paths])
    label = queue[0]
    jpeg = tf.read_file(queue[1])
    image = tf.image.decode_jpeg(jpeg,channels=3)
    image = tf.image.resize_images(image,[256,256])

    image = tf.image.per_image_standardization(image)

    return tf.train.batch([image,label],len(data_list))

In [11]:
def accuracy(logits,labels):
    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    return accuracy
def split_image_and_label(data_list):
    '''
    data_list should be [[data,label]...]
    '''
    data = []
    labels = []
    for value,label in data_list:
        data.append(value)
        labels.append(label)
    return data, labels

In [12]:
# 以下のセルにあるtrain, testは実データなので、読み込みが必要。
def convert_image_path_to_data(path):
    jpeg_r = tf.read_file(path)
    image = tf.image.decode_jpeg(jpeg_r,channels=3)
    image = tf.image.resize_images(image,[256,256])
    image = tf.image.per_image_standardization(image)

    return image


In [ ]:
with tf.Graph().as_default():
    images_placeholder = tf.placeholder("float",shape=(None,IMAGE_PIXELS))
    labels_placeholder = tf.placeholder("float",shape=(None,NUM_CLASSES))
    keep_prob = tf.placeholder("float")
    
    logits = inference(images_placeholder,keep_prob)
    loss_value = loss(logits,labels_placeholder)
    train_op = training(loss_value,learning_rate)
    acc = accuracy(logits,labels_placeholder)

    train_image, train_label = split_image_and_label(train)
    test_image,test_label = split_image_and_label(test)
    
    train_image = [convert_image_path_to_data(path) for path in train_image]
    test_image = [convert_image_path_to_data(path) for path in test_image]
    print("success converting")
    # 保存の準備
    saver = tf.train.Saver()
    # Sessionの作成
    sess = tf.Session()
    # 変数の初期化
    sess.run(tf.initialize_all_variables())
    # TensorBoardで表示する値の設定
#     summary_op = tf.merge_all_summaries()
#     summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph_def)
    print
    # 訓練の実行
    for step in range(max_steps):
        for i in range(int(len(train_image)/BATCH_SIZE)):
            # batch_size分の画像に対して訓練の実行
            batch = BATCH_SIZE*i
            # feed_dictでplaceholderに入れるデータを指定する
            sess.run(train_op, feed_dict={
              images_placeholder: train_image[batch:batch+BATCH_SIZE],
              labels_placeholder: train_label[batch:batch+BATCH_SIZE],
              keep_prob: 0.5})

        # 1 step終わるたびに精度を計算する
        train_accuracy = sess.run(acc, feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        print("step %d, training accuracy %g"%(step, train_accuracy))

        # 1 step終わるたびにTensorBoardに表示する値を追加する
        summary_str = sess.run(summary_op, feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        summary_writer.add_summary(summary_str, step)

# 訓練が終了したらテストデータに対する精度を表示
print("test accuracy %g"%sess.run(acc, feed_dict={
    images_placeholder: test_image,
    labels_placeholder: test_label,
    keep_prob: 1.0}))

# 最終的なモデルを保存
save_path = saver.save(sess, "model.ckpt")

In [13]:
train[10]

(0, '../data/train_fish/YFT/img_01380.jpg')